# Lista 06

## Implementacja APRIORI i jego komponentów

In [1]:
import functools
import itertools

import numpy as np
import pandas as pd
import scipy.sparse

### Wczytywanie używanych później datasetów

In [2]:
def load_dataset(filename):
    rows = []
    with open(filename) as dataset:
        for line in dataset:
            rows.append([int(item) for item in line.split()]) # wiersz 0 i kolumna 0 być może pozostaną puste, trudno
    matrix = scipy.sparse.lil_matrix(
        (1+len(rows), 1+max([max([v for v in r]) for r in rows])), dtype='?')
    print(matrix.shape)
    for idx, r in enumerate(rows):
        matrix[idx, r] = True
    return matrix.tocsc()

In [3]:
DATASETS = {}

### Właściwy algorytm

In [4]:
@functools.lru_cache(maxsize=42_000_000)
def indices_containing(matrix_name, item):
    return set(DATASETS[matrix_name][:, item].nonzero()[0].tolist())

In [5]:
def support(matrix_name, itemset):
    first_element = True
    indices = set()
    for item in itemset:
        this_item_indices = indices_containing(matrix_name, item)
        indices = this_item_indices if first_element else indices & this_item_indices
        first_element = False
    return len(indices)

In [6]:
def gen_frequent_itemsets(matrix_name, min_supp):
    matrix = DATASETS[matrix_name]
    min_supp = min_supp if min_supp == round(min_supp) else min_supp * matrix.shape[0]
    sets = {}
    sets[1] = [
        {col}
        for col in range(matrix.shape[1]) if support(matrix_name, {col}) > min_supp
    ]
    K = 2
    while sets[K-1]:
        print(f'doing K = {K}')
        print(len(sets[K-1]))
        sets[K] = set([
            frozenset(oldset | {newelem})
            for newelem in
                functools.reduce(lambda s1, s2: s1 | s2, sets[K-1])  # flatten
            for oldset in sets[K-1]
        ])
        sets[K] = list(filter(
            lambda newset: 
                support(matrix_name, newset) > min_supp
                and len(newset) == K,
            sets[K]))
        K += 1
    
    return functools.reduce(lambda s1, s2: s1 + s2, sets.values())

In [8]:
def gen_association_rules(matrix_name, itemsets, min_confidence):
    matrix = DATASETS[matrix_name]
    rules = []
    matrix_size = matrix.shape[0]

    for itemset in itemsets:
        itemset = list(itemset)
        antedecent_count = 1
        antedecents = set(itemset[:antedecent_count])
        consequents = set(itemset[antedecent_count:])
        while (confidence := \
            support(matrix_name, full_set := antedecents | consequents) / \
            support(matrix_name, antedecents)) >= min_confidence \
            and antedecent_count < len(full_set):
                
            antedecents = set(itemset[:antedecent_count])
            consequents = set(itemset[antedecent_count:])
            rules.append({
                'rule': f'{antedecents} -> {consequents}',
                'fractional support': support(matrix_name, antedecents | consequents) / matrix_size,
                'confidence': confidence,
                'lift': confidence / (support(matrix_name, consequents) / matrix_size),
                'leverage':
                    (support(matrix_name, antedecents | consequents) / matrix_size) - \
                        (support(matrix_name, antedecents) / matrix_size) * \
                            (support(matrix_name, consequents) / matrix_size)
            })
            antedecent_count += 1
            
    return pd.DataFrame(rules)

In [9]:
def apriori(matrix_name, min_support, min_confidence):
    frequent_itemsets = gen_frequent_itemsets(matrix_name, min_support)
    return gen_association_rules(matrix_name, frequent_itemsets, min_confidence)

## Zestaw danych z artykułu na Wikipedii

In [58]:
wiki_matrix = load_dataset('wiki.dat')

(8, 5)


In [59]:
DATASETS['wiki'] = wiki_matrix

In [62]:
apriori('wiki', 0, 0).sort_values('lift', ascending=False)

doing K = 2
4
doing K = 3
6
doing K = 4
4
doing K = 5
1


,rule,fractional support,confidence,lift,leverage
8,"{1} -> {2, 4}",0.250,0.666667,1.333333,0.062500
3,{1} -> {2},0.375,1.000000,1.333333,0.093750
14,"{1} -> {2, 3, 4}",0.125,0.333333,1.333333,0.031250
0,{3} -> {4},0.375,0.750000,1.200000,0.062500
4,{2} -> {4},0.500,0.666667,1.066667,0.031250
1,{1} -> {4},0.250,0.666667,1.066667,0.015625
9,"{1, 2} -> {4}",0.250,0.666667,1.066667,0.015625
2,{2} -> {3},0.375,0.500000,1.000000,0.000000
6,"{2} -> {3, 4}",0.250,0.333333,0.888889,-0.031250
10,"{1} -> {2, 3}",0.125,0.333333,0.888889,-0.015625


## Zestaw danych RETAIL

In [38]:
retail_matrix = load_dataset('retail.dat')
retail_matrix[:20]

(88163, 16470)


<20x16470 sparse matrix of type '<class 'numpy.bool_'>'
	with 152 stored elements in Compressed Sparse Column format>

In [39]:
DATASETS['retail'] = retail_matrix

In [40]:
retail_res = apriori('retail', 0.02, 0.3)

doing K = 2
355
doing K = 3
388
doing K = 4
169
doing K = 5
29
doing K = 6
4


In [41]:
retail_res_sorted = retail_res.sort_values('confidence', ascending=False)

In [46]:
# retail_res_sorted[abs(retail_res_sorted['leverage']) >= 0.01].head(40)
retail_res_sorted.head(40)

,rule,fractional support,confidence,lift,leverage
121,{371} -> {38},0.008700,0.980818,5.544492,0.007131
64,{105} -> {38},0.007293,0.978691,5.532466,0.005975
84,{170} -> {38},0.034380,0.978057,5.528884,0.028161
173,{3904} -> {38},0.003777,0.976540,5.520304,0.003093
124,{37} -> {38},0.011864,0.973929,5.505548,0.009709
13,{840} -> {38},0.004299,0.969309,5.479433,0.003514
70,{370} -> {38},0.004106,0.965333,5.456956,0.003354
154,{56} -> {38},0.005830,0.960748,5.431033,0.004757
49,{281} -> {38},0.004900,0.953642,5.390868,0.003991
118,{36} -> {38},0.031646,0.950272,5.371818,0.025755


## Zestaw danych KOSARAK

In [9]:
kosarak_matrix = load_dataset('kosarak.dat')
kosarak_matrix

(990003, 41271)


<990003x41271 sparse matrix of type '<class 'numpy.bool_'>'
	with 8018988 stored elements in Compressed Sparse Column format>

In [10]:
DATASETS['kosarak'] = kosarak_matrix

In [11]:
kosarak_res = apriori('kosarak', 0.2, 0.3)

doing K = 2
70
doing K = 3
187
doing K = 4
186
doing K = 5
80
doing K = 6
16
doing K = 7
1


In [12]:
kosarak_res.sort_values('confidence', ascending=False).head(50)

,rule,fractional support,confidence,lift,leverage
81,{438} -> {6},0.018566,0.971973,1.600095,0.006963
66,{254} -> {6},0.009842,0.966954,1.591834,0.003659
24,{85} -> {6},0.008595,0.966822,1.591616,0.003195
45,{364} -> {6},0.011874,0.966138,1.590490,0.004408
62,{90} -> {6},0.021503,0.964917,1.588480,0.007966
43,{25} -> {6},0.012341,0.964477,1.587757,0.004569
29,{229} -> {6},0.009775,0.964421,1.587664,0.003618
3,{155} -> {6},0.011178,0.962763,1.584935,0.004125
71,{85} -> {7},0.008558,0.962618,10.966818,0.007777
78,{379} -> {6},0.009508,0.962179,1.583973,0.003505


## Zestaw danych Ta-Feng

In [10]:
tafeng_columns = [
    'timestamp', 'customer', 'age_group', 'zipcode_area',
    'prod_subclass', 'prod_id', 'prod_amount', 'asset', 'sales_price'
]
TAFENG_TRANSFORMATORS = {}

In [11]:
tafeng_11 = pd.read_csv('D11', sep=';', names=tafeng_columns)
tafeng_12 = pd.read_csv('D12', sep=';', names=tafeng_columns)
tafeng_01 = pd.read_csv('D01', sep=';', names=tafeng_columns)
tafeng_02 = pd.read_csv('D02', sep=';', names=tafeng_columns)

taf = [tafeng_11, tafeng_12, tafeng_01, tafeng_02]

for idx, _ in enumerate(taf):
    # przy mojej implementacji przy użyciu sparse matrixów te wartości są po prostu za duże
    max_subclass_id = max(taf[idx]['prod_subclass'].unique())
    TAFENG_TRANSFORMATORS[idx] = {
        k: 100+max_subclass_id+v for v, k in enumerate(taf[idx]['prod_id'].unique())
    }

    taf[idx] = taf[idx].replace({'prod_id': TAFENG_TRANSFORMATORS[idx]})
    taf[idx] = taf[idx].groupby(['timestamp', 'customer'], as_index=False).agg(
        lambda cell: 
            cell.tolist() if cell.name not in ('age_group', 'zipcode_area') else cell.tolist()[0])

taf[0]

,timestamp,customer,age_group,zipcode_area,prod_subclass,prod_id,prod_amount,asset,sales_price
0,2000-11-01 00:00:00,38317,J,E,"[130315, 120105]","[780611, 780783]","[2, 1]","[56, 28]","[48, 28]"
1,2000-11-01 00:00:00,45902,H,E,"[100304, 130204, 100511, 100113]","[781334, 780814, 781339, 781354]","[1, 1, 6, 1]","[24, 114, 210, 112]","[28, 119, 313, 95]"
2,2000-11-01 00:00:00,45957,G,E,[110217],[780788],[1],[180],[133]
3,2000-11-01 00:00:00,46855,D,E,"[110411, 110401, 110117]","[780610, 780674, 780804]","[3, 1, 1]","[51, 43, 77]","[57, 39, 89]"
4,2000-11-01 00:00:00,58698,D,E,"[130201, 130204, 110401, 130206, 120109, 120109]","[781090, 780985, 781008, 781099, 781103, 781123]","[1, 1, 1, 1, 1, 1]","[45, 48, 53, 47, 36, 25]","[53, 54, 64, 54, 48, 33]"
...,...,...,...,...,...,...,...,...,...
31855,2000-11-30 00:00:00,2161983,H,G,"[110117, 130404, 130206, 110105, 110208, 76013...","[780844, 784650, 781000, 786956, 792431, 78432...","[1, 6, 1, 1, 1, 4, 1, 1, 1, 2, 2, 1]","[41, 342, 34, 28, 14, 256, 64, 42, 35, 28, 56,...","[43, 354, 40, 34, 19, 236, 68, 48, 39, 18, 68,..."
31856,2000-11-30 00:00:00,2162003,C,C,"[720221, 500205, 500202, 500201]","[793021, 784070, 781914, 780896]","[1, 1, 4, 1]","[93, 51, 432, 150]","[98, 62, 460, 156]"
31857,2000-11-30 00:00:00,2162027,F,C,"[100413, 520144, 520211, 520213, 500316, 10041...","[796036, 788347, 791911, 788664, 789423, 78414...","[1, 1, 1, 1, 1, 1, 1, 1, 2]","[93, 26, 48, 24, 109, 100, 420, 80, 28]","[99, 39, 70, 37, 149, 109, 550, 119, 36]"
31858,2000-11-30 00:00:00,2162034,D,F,"[100508, 100414, 100414]","[781113, 781613, 781333]","[6, 3, 3]","[72, 267, 243]","[66, 285, 285]"


In [12]:
for idx, _ in enumerate(taf):
    for num, letter in enumerate(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']):
        taf[idx].loc[taf[idx]['age_group'] == letter + ' ', 'age_group']= -1 - num
        if letter not in ('I', 'J', 'K'):  # występują tylko w wieku
            taf[idx].loc[taf[idx]['zipcode_area'] == letter + ' ', 'zipcode_area'] = -42 - num

In [13]:
for idx, _ in enumerate(taf):
    taf[idx] = taf[idx].drop(['timestamp', 'customer', 'prod_amount', 'asset', 'sales_price'], 1)

taf[0]

,age_group,zipcode_area,prod_subclass,prod_id
0,-10,-46,"[130315, 120105]","[780611, 780783]"
1,-8,-46,"[100304, 130204, 100511, 100113]","[781334, 780814, 781339, 781354]"
2,-7,-46,[110217],[780788]
3,-4,-46,"[110411, 110401, 110117]","[780610, 780674, 780804]"
4,-4,-46,"[130201, 130204, 110401, 130206, 120109, 120109]","[781090, 780985, 781008, 781099, 781103, 781123]"
...,...,...,...,...
31855,-8,-48,"[110117, 130404, 130206, 110105, 110208, 76013...","[780844, 784650, 781000, 786956, 792431, 78432..."
31856,-3,-44,"[720221, 500205, 500202, 500201]","[793021, 784070, 781914, 780896]"
31857,-6,-44,"[100413, 520144, 520211, 520213, 500316, 10041...","[796036, 788347, 791911, 788664, 789423, 78414..."
31858,-4,-47,"[100508, 100414, 100414]","[781113, 781613, 781333]"


In [14]:
taf_to_name = ['11', '12', '01', '02']

In [15]:
for idx, _ in enumerate(taf):
    csv = taf[idx].to_csv(sep=' ', header=False, index=False)
    csv = csv.replace('"', '').replace('[', '').replace(']', '').replace(',', '')

    with open(f"tafeng_{taf_to_name[idx]}.csv", 'w') as f:
        f.write(csv)

In [16]:
for idx, _ in enumerate(taf):
    curr_matrix = load_dataset(f'tafeng_{taf_to_name[idx]}.csv')
    print(curr_matrix.__repr__())
    DATASETS[f'tafeng#{idx}'] = curr_matrix

(31861, 798166)
<31861x798166 sparse matrix of type '<class 'numpy.bool_'>'
	with 479425 stored elements in Compressed Sparse Column format>
(26767, 797294)
<26767x797294 sparse matrix of type '<class 'numpy.bool_'>'
	with 384690 stored elements in Compressed Sparse Column format>
(29902, 798470)
<29902x798470 sparse matrix of type '<class 'numpy.bool_'>'
	with 460202 stored elements in Compressed Sparse Column format>
(31052, 797478)
<31052x797478 sparse matrix of type '<class 'numpy.bool_'>'
	with 432370 stored elements in Compressed Sparse Column format>


In [24]:
def reverse_transform(rev_transformer, s):
    for k, v in rev_transformer.items():
        s = s.replace(str(k), str(v))
    return s

### Listopad

In [30]:
tafeng_11_res = apriori('tafeng#0', 0.02, 0.3)
reverse_transformer = {v: k for k, v in TAFENG_TRANSFORMATORS[0].items()}
tafeng_11_res['rule'] = tafeng_11_res['rule'].map(
    lambda rule: reverse_transform(reverse_transformer, rule))

tafeng_11_res.sort_values('confidence', ascending=False)

doing K = 2
88
doing K = 3
67
doing K = 4
5


,rule,fractional support,confidence,lift,leverage
22,{4710421090059} -> {100508},0.028248,1.000000,28.220549,0.027247
10,{4711080010112} -> {130206},0.020464,1.000000,10.782064,0.018566
14,{110106} -> {4711271000014},0.074040,0.897982,12.128283,0.067936
6,{130315} -> {4714981010038},0.107624,0.860261,7.993226,0.094159
11,{2270022005125} -> {4973167711156},0.027306,0.790909,13.290693,0.025252
20,{500210} -> {4710908131589},0.030759,0.600122,19.510716,0.029182
19,{20222406} -> {20386221},0.052760,0.465780,1.107727,0.005131
2,{100312} -> {100205},0.026867,0.457754,3.496644,0.019183
5,{100312} -> {20386221},0.024011,0.409091,0.972908,-0.000669
4,{130106} -> {20488109},0.022630,0.356225,1.202043,0.003804


### Grudzień

In [31]:
tafeng_12_res = apriori('tafeng#1', 0.02, 0.3)
reverse_transformer = {v: k for k, v in TAFENG_TRANSFORMATORS[1].items()}
tafeng_12_res['rule'] = tafeng_12_res['rule'].map(
    lambda rule: reverse_transform(reverse_transformer, rule))

tafeng_12_res.sort_values('confidence', ascending=False)

doing K = 2
77
doing K = 3
52
doing K = 4
2


,rule,fractional support,confidence,lift,leverage
7,{4713985863121} -> {130315},0.028132,1.000000,18.460000,0.026608
15,{4711271000014} -> {110106},0.095117,1.000000,9.055142,0.084613
11,{20562472} -> {4714995000124},0.030299,0.808574,12.928977,0.027955
16,{530301} -> {4710683100015},0.022864,0.522184,22.838737,0.021863
8,{4710215200022} -> {4711180102021},0.054433,0.468790,1.144170,0.006859
2,{100312} -> {100205},0.022079,0.421541,3.306969,0.015403
20,"{4711271000014} -> {4711180102021, 110106}",0.039452,0.414768,9.221015,0.035173
21,"{4711271000014, 110106} -> {4711180102021}",0.039452,0.414768,1.012319,0.000480
6,{4711271000014} -> {4711180102021},0.039452,0.414768,1.012319,0.000480
4,{110117} -> {4710063031106},0.027310,0.381126,13.955683,0.025353


### Styczeń

In [32]:
tafeng_01_res = apriori('tafeng#2', 0.02, 0.3)
reverse_transformer = {v: k for k, v in TAFENG_TRANSFORMATORS[2].items()}
tafeng_01_res['rule'] = tafeng_01_res['rule'].map(
    lambda rule: reverse_transform(reverse_transformer, rule))

tafeng_01_res.sort_values('confidence', ascending=False)

doing K = 2
99
doing K = 3
47
doing K = 4
1


,rule,fractional support,confidence,lift,leverage
2,{4712425010712} -> {500201},0.028326,1.000000,10.949103,0.025739
3,{4712425010255} -> {500210},0.020701,1.000000,23.324493,0.019813
8,{4714981010038} -> {130315},0.024079,1.000000,20.396999,0.022898
7,{76174644548} -> {41554616033},0.029998,0.810298,12.368317,0.027573
4,{110217} -> {4719090900065},0.054913,0.448022,8.158799,0.048182
0,{120103} -> {4710011401128},0.020601,0.321503,15.606472,0.019281
6,{8888240001160} -> {4711629910521},0.060966,0.314094,1.140502,0.007511
9,{9300697100139} -> {4711629910521},0.050632,0.306478,1.112847,0.005134
10,{110401} -> {4711629910521},0.021905,0.306218,1.111904,0.002205
1,{110217} -> {4711629910521},0.037489,0.305866,1.110627,0.003734


### Luty

In [33]:
tafeng_02_res = apriori('tafeng#3', 0.02, 0.3)
reverse_transformer = {v: k for k, v in TAFENG_TRANSFORMATORS[3].items()}
tafeng_02_res['rule'] = tafeng_02_res['rule'].map(
    lambda rule: reverse_transform(reverse_transformer, rule))

tafeng_02_res.sort_values('confidence', ascending=False)

doing K = 2
87
doing K = 3
58
doing K = 4
4


,rule,fractional support,confidence,lift,leverage
0,{4711271000014} -> {110106},0.027985,1.000000,22.534107,0.026743
6,{4714981010038} -> {130315},0.122794,1.000000,6.963893,0.105161
11,{4710114128038} -> {500201},0.022704,1.000000,10.385284,0.020518
13,{4711856000088} -> {110501},0.020611,1.000000,22.501449,0.019695
14,{4711856000125} -> {110501},0.028919,1.000000,22.501449,0.027634
8,{4710223169137} -> {4710076115473},0.030529,0.807496,12.663817,0.028119
2,{100312} -> {100205},0.032043,0.494779,4.040997,0.024114
1,{500202} -> {4710908110232},0.022800,0.381466,16.730603,0.021438
10,{500202} -> {4710114362029},0.020578,0.344289,16.730603,0.019348
9,{110217} -> {4710265849066},0.027599,0.342116,12.396008,0.025372


## Scratchpad

In [97]:
retail_frequent_itemsets = gen_frequent_itemsets(retail_matrix, 0.02)
retail_frequent_itemsets

doing K = 2
355
doing K = 3
388
doing K = 4
169
doing K = 5
29
doing K = 6
4


[{2},
 {9},
 {10},
 {11},
 {12},
 {15},
 {18},
 {19},
 {23},
 {30},
 {31},
 {32},
 {36},
 {37},
 {38},
 {39},
 {41},
 {45},
 {47},
 {48},
 {49},
 {52},
 {53},
 {55},
 {56},
 {60},
 {62},
 {65},
 {76},
 {78},
 {79},
 {80},
 {89},
 {94},
 {96},
 {101},
 {103},
 {105},
 {107},
 {110},
 {117},
 {123},
 {136},
 {147},
 {150},
 {155},
 {156},
 {161},
 {164},
 {170},
 {175},
 {176},
 {178},
 {179},
 {185},
 {186},
 {195},
 {201},
 {208},
 {209},
 {225},
 {229},
 {232},
 {237},
 {242},
 {246},
 {248},
 {249},
 {251},
 {255},
 {258},
 {259},
 {260},
 {261},
 {264},
 {267},
 {269},
 {270},
 {271},
 {272},
 {279},
 {281},
 {286},
 {297},
 {301},
 {310},
 {319},
 {334},
 {338},
 {340},
 {341},
 {345},
 {346},
 {351},
 {352},
 {365},
 {370},
 {371},
 {374},
 {379},
 {384},
 {389},
 {390},
 {396},
 {398},
 {405},
 {407},
 {408},
 {413},
 {415},
 {418},
 {420},
 {424},
 {425},
 {426},
 {432},
 {438},
 {441},
 {449},
 {464},
 {475},
 {479},
 {488},
 {490},
 {497},
 {514},
 {522},
 {533},
 {535},
 {544

In [ ]:
rules = gen_association_rules('retail', retail_frequent_itemsets, 0.3)